In [ ]:
%reset -f

In [ ]:
!pip install pyspark --quiet

In [ ]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("Basics")
sc = SparkContext(conf=conf)

In [ ]:
lines = sc.textFile("/content/drive/MyDrive/DVI/DVI-10/SecA/data/u_data.txt")

In [ ]:
'userid,movieid,rating,date'

'userid,movieid,rating,date'

In [ ]:
lines.map(lambda x:(x.split(",")[1],1)).top(5)

[('999', 1), ('999', 1), ('999', 1), ('999', 1), ('999', 1)]

In [ ]:
import pandas as pd
pandasDF = pd.read_csv("https://raw.githubusercontent.com/AIP-codedb/public/main/data/u_data.txt")
pandasDF.head()

,196,242,3,881250949
0,186,302,3.0,891717742.0
1,22,377,1.0,878887116.0
2,244,51,2.0,880606923.0
3,166,346,1.0,886397596.0
4,298,474,4.0,884182806.0


In [ ]:
import pandas as pd
books = pd.read_csv("https://raw.githubusercontent.com/AIP-codedb/public/main/data/books.csv",error_bad_lines=False)
books.head()

<ipython-input-7-184c13e56129>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv("https://raw.githubusercontent.com/AIP-codedb/public/main/data/books.csv",error_bad_lines=False)
Skipping line 4012: expected 10 fields, saw 11
Skipping line 5688: expected 10 fields, saw 11
Skipping line 7056: expected 10 fields, saw 11
Skipping line 10601: expected 10 fields, saw 11
Skipping line 10668: expected 10 fields, saw 11



,bookID,title,authors,average_rating,isbn,isbn13,language_code,# num_pages,ratings_count,text_reviews_count
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling-Mary GrandPré,4.56,0439785960,9780439785969,eng,652,1944099,26249
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling-Mary GrandPré,4.49,0439358078,9780439358071,eng,870,1996446,27613
2,3,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling-Mary GrandPré,4.47,0439554934,9780439554930,eng,320,5629932,70390
3,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.41,0439554896,9780439554893,eng,352,6267,272
4,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling-Mary GrandPré,4.55,043965548X,9780439655484,eng,435,2149872,33964


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count
spark = (SparkSession.builder.appName("DF").getOrCreate())
books = spark.createDataFrame(books)


In [ ]:
books.createOrReplaceTempView("books")

In [ ]:
spark.sql("""

select language_code,count(*) as cnt
from books
group by language_code
order by cnt DESC

""").show(5)

+-------------+-----+
|language_code|  cnt|
+-------------+-----+
|          eng|10594|
|        en-US| 1699|
|          spa|  419|
|        en-GB|  341|
|          ger|  238|
+-------------+-----+
only showing top 5 rows



In [ ]:
movies = spark.read.format("csv").load("/content/drive/MyDrive/DVI/DVI-10/SecA/data/u_data.txt")


In [ ]:
movies.columns

['_c0', '_c1', '_c2', '_c3']

In [ ]:
movies.createOrReplaceTempView("movies_table")

In [ ]:
spark.sql('''
select _c1,count(*) as cnt
from movies_table
group by _c1
order by cnt DESC
''').show(10)

+---+---+
|_c1|cnt|
+---+---+
| 50|301|
|258|279|
|100|268|
|288|266|
|181|259|
|286|241|
|294|239|
|  1|225|
|  7|220|
| 56|220|
+---+---+
only showing top 10 rows



In [ ]:
movies.select("_c1").groupBy("_c1").count().orderBy("count",ascending=False).show(10)

+---+-----+
|_c1|count|
+---+-----+
| 50|  301|
|258|  279|
|100|  268|
|288|  266|
|181|  259|
|286|  241|
|294|  239|
|  1|  225|
|  7|  220|
| 56|  220|
+---+-----+
only showing top 10 rows



In [ ]:
lines = sc.textFile("/content/drive/MyDrive/DVI/DVI-10/SecA/data/u_data.txt")

In [ ]:
movies = lines.map(lambda x:(int(x.split(",")[1]),1))

In [ ]:
movies.top(5)

[(1661, 1), (1660, 1), (1657, 1), (1656, 1), (1655, 1)]

In [ ]:
moviesCounts = movies.reduceByKey(lambda x,y:x+y)
flipped = moviesCounts.map(lambda x:(x[1],x[0]))
sortedMovies = flipped.sortByKey(ascending = False)
mv = sortedMovies.collect()


In [ ]:
type(mv)

list

SyntaxError: invalid syntax (<ipython-input-25-61d5a088915b>, line 1)